In [1]:
#last update march 6 8pm
import os
import numpy as np
import random
import math
import json

from functools import partial

## Imports for plotting
import matplotlib.pyplot as plt
plt.set_cmap('cividis')
#%matplotlib inline
from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

## tqdm for loading bars
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
from torch.types import _size

## Torchvision
import torchvision
from torchvision.datasets import CIFAR100
from torchvision import transforms


## Audio
from transformers import AutoProcessor, ASTModel
from datasets import load_dataset
from transformers import AutoProcessor, ASTModel

####image
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPVisionModel , CLIPVisionModelWithProjection
torch.cuda.empty_cache()
import gc
gc.collect()
from transformers import AutoTokenizer, CLIPTextModel,AutoModel


2024-08-13 17:44:11.316970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<Figure size 640x480 with 0 Axes>

In [2]:
from model import Transformer
#from dataset import BilingualDataset, causal_mask
from config import get_config, get_weights_file_path, latest_weights_file_path

#import torchtext.datasets as datasets
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import LambdaLR

import warnings
from tqdm import tqdm
import os
from pathlib import Path

# Huggingface datasets and tokenizers
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

import torchmetrics

from torch.utils.tensorboard import SummaryWriter

from datasets import Audio
from datasets import Dataset, Image
from numpy.linalg import norm
import json
import os

import cv2
import pandas as pd
import torch

#from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoImageProcessor, AutoModelForImageClassification
from transformers import CLIPProcessor

import config

<Figure size 640x480 with 0 Axes>

In [ ]:

import config
# config the database  
def get_ds():
    #path= path for the imagenet dataset
    filepathlabel='/path/magenet_100/archive/Labels.json'
   ## filepatht1='/home_nfs/mranga/imagenet_100/rchive/train.X1'
    #filepatht2='/home_nfs/mranga/imagenet_100/archive/train.X2'
   # filepatht3='/home_nfs/mranga/imagenet_100/archive/train.X3'
   # filepatht4='/home_nfs/mranga/imagenet_100/archive/train.X4'
    #filepathval='/media/maithri/ee0c61c0-1e77-4746-ae07-ddc0cd940859/dataset/imagenet_100/archive/train.X1'
    with open(filepathlabel, 'r') as f:
         labels = json.load(f)
    #print(labels)
    images = []
    #/media/maithri/ee0c61c0-1e77-4746-ae07-ddc0cd940859/dataset/imagenet_100
    for label in os.listdir('/path/imagenet_100/archive/train.X1'):
        for filename in os.listdir(f'/path/imagenet_100/archive/train.X1/{label}'):
             images.append({
                 'path': f'/path/magenet_100/archive/train.X1/{label}/{filename}',
                 'label': labels[label],
                 'id': label,
             })
    for label in os.listdir('/path/imagenet_100/archive/train.X2'):
        for filename in os.listdir(f'/path/imagenet_100/archive/train.X2/{label}'):
             images.append({
                 'path': f'/path/imagenet_100/archive/train.X2/{label}/{filename}',
                 'label': labels[label],
                 'id': label,
             })
    for label in os.listdir('/path/magenet_100/archive/train.X3'):
        for filename in os.listdir(f'/path/magenet_100/archive/train.X3/{label}'):
             images.append({
                 'path': f'/path/magenet_100/archive/train.X3/{label}/{filename}',
                 'label': labels[label],
                 'id': label,
             })
    for label in os.listdir('/path/imagenet_100/archive/train.X4'):
        for filename in os.listdir(f'/path/magenet_100/archive/train.X4/{label}'):
              images.append({
                 'path': f'/path/imagenet_100/archive/train.X4/{label}/{filename}',
                 'label': labels[label],
                 'id': label,
             })

    train_dataloader = pd.DataFrame(images)
    
    val_images = []

    for label in os.listdir('/path/imagenet_100/archive/val.X'):
        for filename in os.listdir(f'/path/dataset/imagenet_100/archive/val.X/{label}'):
             val_images.append({
                 'path': f'/path/dataset/imagenet_100/archive/val.X/{label}/{filename}',
                 'label': labels[label],
                 'id': label,
              })

    val_dataloader = pd.DataFrame(val_images)


    #train_ds_size=int(0.9*len(dataset))
    #val_ds_size= len(dataset) - train_ds_size
    #train_dataset_both ,val_dataset_both = random_split(dataset ,[train_ds_size, val_ds_size])
    batch_size=config['batch_size'] 
    train_dataloader = DataLoader(images ,batch_size , shuffle=True)
    val_dataloader = DataLoader(val_images , batch_size=4, shuffle=True)
    
    return train_dataloader,val_dataloader,labels


"\nimport config\n# config the database  \ndef get_ds():\n    filepathlabel='/home_nfs/mranga/imagenet_100/archive/Labels.json'\n    filepatht1='/home_nfs/mranga/imagenet_100/rchive/train.X1'\n    filepatht2='/home_nfs/mranga/imagenet_100/archive/train.X2'\n    filepatht3='/home_nfs/mranga/imagenet_100/archive/train.X3'\n    filepatht4='/home_nfs/mranga/imagenet_100/archive/train.X4'\n    #filepathval='/home_nfs/mranga/imagenet_100/imagenet_100/archive/train.X1'\n    with open(filepathlabel, 'r') as f:\n         labels = json.load(f)\n\n    images = []\n\n    for label in os.listdir('/home_nfs/mranga/imagenet_100/archive/train.X1'):\n        for filename in os.listdir(f'/home_nfs/mranga/imagenet_100/archive/train.X1/{label}'):\n             images.append({\n                 'path': f'/home_nfs/mranga/imagenet_100/archive/train.X1/{label}/{filename}',\n                 'label': labels[label],\n                 'id': label,\n             })\n    for label in os.listdir('/home_nfs/mranga/

In [5]:
def imageNet100id():
    
   imagenet_id=["n01968897",
    "n01770081",
    "n01818515",
    "n02011460",
    "n01496331",
    "n01847000",
    "n01687978",
    "n01740131",
    "n01537544",
    "n01491361",
    "n02007558",
    "n01735189",
    "n01630670",
    "n01440764",
    "n01819313",
    "n02002556",
    "n01667778",
    "n01755581",
    "n01924916",
    "n01751748",
    "n01984695",
    "n01729977",
    "n01614925",
    "n01608432",
    "n01443537",
    "n01770393",
    "n01855672",
    "n01560419",
    "n01592084",
    "n01914609",
    "n01582220",
    "n01667114",
    "n01985128",
    "n01820546",
    "n01773797",
    "n02006656",
    "n01986214",
    "n01484850",
    "n01749939",
    "n01828970",
    "n02018795",
    "n01695060",
    "n01729322",
    "n01677366",
    "n01734418",
    "n01843383",
    "n01806143",
    "n01773549",
    "n01775062",
    "n01728572",
    "n01601694",
    "n01978287",
    "n01930112",
    "n01739381",
    "n01883070",
    "n01774384",
    "n02037110",
    "n01795545",
    "n02027492",
    "n01531178",
    "n01944390",
    "n01494475",
    "n01632458",
    "n01698640",
    "n01675722",
    "n01877812",
    "n01622779",
    "n01910747",
    "n01860187",
    "n01796340",
    "n01833805",
    "n01685808",
    "n01756291",
    "n01514859",
    "n01753488",
    "n02058221",
    "n01632777",
    "n01644900",
    "n02018207",
    "n01664065",
    "n02028035",
    "n02012849",
    "n01776313",
    "n02077923",
    "n01774750",
    "n01742172",
    "n01943899",
    "n01798484",
    "n02051845",
    "n01824575",
    "n02013706",
    "n01955084",
    "n01773157",
    "n01665541",
    "n01498041",
    "n01978455",
    "n01693334",
    "n01950731",
    "n01829413",
    "n01514668"]
   return imagenet_id


In [6]:
#saas,sasas,lables=get_ds()

In [7]:

def get_model(): 
    model = Transformer(config["d_model"], config["ffn_hidden"], config["num_heads"], config["drop_prob"], config["num_layers"], config["kn_vocab_size"])
    return model

In [8]:
def get_image_model():

    image_model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32",output_hidden_states =True,output_attentions=True)
    image_processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
    #image_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    return image_model,image_processor 

In [9]:
def get_audio_model():

    processor_audio = AutoProcessor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
    model_audio = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593", output_hidden_states =True,output_attentions=True)
    

    return model_audio,processor_audio  

In [10]:
def get_text_model():
    text_model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2",output_hidden_states =True,output_attentions=True)
    text_Tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
    return text_model ,text_Tokenizer 

In [11]:
def get_hidden_layer():
    
    image_layerno_1=7
    image_layerno_2=6
    print("image layer number",image_layerno_1,image_layerno_2)
    return image_layerno_1 ,image_layerno_2

In [ ]:

def run_validation(model, validation_ds,labels_all, device, print_msg, global_step, writer, num_examples=10):
    model.eval()
    count = 0
    image_model,image_processor =get_image_model()
    model_audio,processor_audio =get_audio_model()
    text_model,text_Tokenizer =get_text_model()
    image100id=imageNet100id()
    image_layerno_1,image_layerno_2= get_hidden_layer()

    source_image = []
    expected = []
    predicted = []
    realclass_count=0
    number_inputs=0
    classid=''
    try:
        # get the console window width
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        # If we can't get the console width, use 80 as default
        console_width = 80

    preds = []

    #batch_size=config['batch_size'] 
    batch_size=1
   # batch_iterator = tqdm(validation_ds.iter(batch_size))
    batch_iterator = tqdm(validation_ds)
       # print("batch_iterator,batch_iterator",batch_iterator)
        #for index, row in df.iterrows():
        #for batch in batch_iterator:




    with torch.no_grad():
        for batch in batch_iterator :
            count += 1

########################  image KQV extraction 

            paths=batch["path"]
            image_file = Dataset.from_dict({"image": paths}).cast_column("image", Image())  
            
            image_model=image_model.to(device)

                
            image_1 = image_processor(images=image_file[0]["image"], return_tensors="pt").to(device)
            image_outputs_1 = image_model(**image_1)
            image_last_hidden_state_1 = image_outputs_1.last_hidden_state
            image_pooled_output_1 = image_outputs_1.pooler_output  
            image_layear_output_1=image_outputs_1.hidden_states[image_layerno_1],image_outputs_1.hidden_states[image_layerno_2]
                  
                  
            image_2 = image_processor(images=image_file[1]["image"], return_tensors="pt").to(device)
            image_model=image_model.to(device)
            image_outputs_2 = image_model(**image_2)
            image_last_hidden_state_2 = image_outputs_2.last_hidden_state
            image_pooled_output_2 = image_outputs_2.pooler_output  
            image_layear_output_2=image_outputs_2.hidden_states[image_layerno_1],image_outputs_2.hidden_states[image_layerno_2]
            
            
            image_3 = image_processor(images=image_file[2]["image"], return_tensors="pt").to(device)
            image_model=image_model.to(device)
            image_outputs_3 = image_model(**image_3)
            image_last_hidden_state_3 = image_outputs_3.last_hidden_state
            image_pooled_output_3 = image_outputs_3.pooler_output  
            image_layear_output_3=image_outputs_3.hidden_states[image_layerno_1],image_outputs_3.hidden_states[image_layerno_2]
            
            image_4 = image_processor(images=image_file[3]["image"], return_tensors="pt").to(device)
            image_model=image_model.to(device)
            image_outputs_4 = image_model(**image_4)
            image_last_hidden_state_4 = image_outputs_4.last_hidden_state
            image_pooled_output_4 = image_outputs_4.pooler_output  
            image_layear_output_4=image_outputs_4.hidden_states[image_layerno_1],image_outputs_4.hidden_states[image_layerno_2]


            label=batch["label"]
          


            inputs_text = text_Tokenizer(text=label[0], padding=True, return_tensors="pt").to(device)
            text_model= text_model.to(device)
            text_outputs = text_model(**inputs_text)
            text_pooled_output = text_outputs.pooler_output

###########################################################################

  
#############################################################################
            text_query_bias=text_model.encoder.layer[image_layerno_1].attention.attn.q.bias,text_model.encoder.layer[image_layerno_2].attention.attn.q.bias 
            text_key_bias=text_model.encoder.layer[image_layerno_1].attention.attn.k.bias,text_model.encoder.layer[image_layerno_2].attention.attn.k.bias
            text_value_bias=text_model.encoder.layer[image_layerno_1].attention.attn.v.bias,text_model.encoder.layer[image_layerno_2].attention.attn.v.bias
            _start = 0
            _end = config["d_model"]
            text_query_weight=text_model.encoder.layer[image_layerno_1].attention.attn.q.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.q.weight[_start:_end, :]
            text_key_weight=text_model.encoder.layer[image_layerno_1].attention.attn.k.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.k.weight[_start:_end, :]
            text_value_weight=text_model.encoder.layer[image_layerno_1].attention.attn.v.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.v.weight[_start:_end, :]



            #image model parameters 
            image_query_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.q_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.q_proj.bias
            image_key_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.k_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.k_proj.bias
            image_value_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.v_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.v_proj.bias

            image_query_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.q_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.q_proj.weight[_start:_end, :]
            image_key_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.k_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.k_proj.weight[_start:_end, :]
            image_value_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.v_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.v_proj.weight[_start:_end, :]

            
            scores_1 = [None] * 100
            scores_2 = [None] * 100
            scores_3 = [None] * 100
            scores_4 = [None] * 100
                  
            for x in range(0, 100):
                #print("text",labels_all[image100id[x]])
                inputs_text = text_Tokenizer(text=labels_all[image100id[x]], padding=True, return_tensors="pt").to(device)
                text_model= text_model.to(device)
                text_outputs = text_model(**inputs_text)
                text_pooled_output = text_outputs.pooler_output

                text_layear_output=text_outputs.hidden_states[image_layerno_1],text_outputs.hidden_states[image_layerno_2]
                proj_output_1=model(batch_size,text_query_bias,text_key_bias,text_value_bias,text_query_weight,text_key_weight,text_value_weight,image_query_bias, image_key_bias,image_value_bias,
                                    image_query_weight,image_key_weight,image_value_weight, text_layear_output, image_layear_output_1, image_pooled_output_1).to(device)
                proj_output_1 = proj_output_1.detach().cpu().numpy()
                text_pooled_output =text_pooled_output.detach().cpu().numpy()
      
                scores_1[x] = np.dot(proj_output_1, text_pooled_output.T)/(norm(proj_output_1)*(norm(text_pooled_output))) 
                    
                    
                proj_output_2=model(batch_size,text_query_bias,text_key_bias,text_value_bias,text_query_weight,text_key_weight,text_value_weight,image_query_bias, image_key_bias,image_value_bias,
                                    image_query_weight,image_key_weight,image_value_weight, text_layear_output, image_layear_output_2, image_pooled_output_2).to(device)  
                      
                    
                proj_output_2 = proj_output_2.detach().cpu().numpy()
                  
                scores_2[x] = np.dot(proj_output_2, text_pooled_output.T)/(norm(proj_output_2)*(norm(text_pooled_output))) 
                
                proj_output_3=model(batch_size,text_query_bias,text_key_bias,text_value_bias,text_query_weight,text_key_weight,text_value_weight,image_query_bias, image_key_bias,image_value_bias,
                                    image_query_weight,image_key_weight,image_value_weight, text_layear_output, image_layear_output_3, image_pooled_output_3).to(device)  
                      
                    
                proj_output_3 = proj_output_3.detach().cpu().numpy()
                  
                scores_3[x] = np.dot(proj_output_3, text_pooled_output.T)/(norm(proj_output_3)*(norm(text_pooled_output))) 
                    #scores_2[x]
                    
                proj_output_4=model(batch_size,text_query_bias,text_key_bias,text_value_bias,text_query_weight,text_key_weight,text_value_weight,image_query_bias, image_key_bias,image_value_bias,
                                    image_query_weight,image_key_weight,image_value_weight, text_layear_output, image_layear_output_4, image_pooled_output_4).to(device)  
                      
                    
                proj_output_4 = proj_output_4.detach().cpu().numpy()
                  
                scores_4[x] = np.dot(proj_output_4, text_pooled_output.T)/(norm(proj_output_4)*(norm(text_pooled_output))) 
                    
                    
                  #print("real_class",np.argmax(scores_1),np.argmax(scores_2))
            real_class=np.argmax(scores_1),np.argmax(scores_2),np.argmax(scores_3),np.argmax(scores_4)
                
           # print("real_class",real_class[0],real_class[1],real_class[2],real_class[3])
                  #print("scores",np.argmax(appfor, axis=0),label)
                
              
        
            classid=str(batch["id"][0]),str(batch["id"][1]),str(batch["id"][2]),str(batch["id"][3])
           # print(str(batch["id"][0]),str(batch["id"][1]),str(batch["id"][2]),str(batch["id"][3]))
           # print(image100id[real_class[0]],image100id[real_class[1]],image100id[real_class[2]],image100id[real_class[3]])
            for x in range(0, 4):
              #  print(x)
                if classid[x] == image100id[real_class[x]]:
                        realclass_count=realclass_count+1
                        number_inputs=number_inputs+1
                        #print("correct",realclass_count)
                else:
                        #  print("wwwwwwwwwww",number_inputs)
                        number_inputs=number_inputs+1


                  

                  #source_image.append(scores)
            expected.append(1.00)
            predicted.append(scores_1)
            


    Accuracy =(realclass_count / number_inputs*100 )

    print("Accuracy", Accuracy,realclass_count,number_inputs, "%")
    return Accuracy 



In [ ]:

def train_model(config):
    device = "cuda" if torch.cuda.is_available() else "mps" if torch.has_mps or torch.backends.mps.is_available() else "cpu"
   #device="cpu"
    print("Using device:", device)
    torch.autograd.set_detect_anomaly(True)
    # Make sure the weights folder exists
    Path(f"{config['model_folder']}").mkdir(parents=True, exist_ok=True)

    train_dataloader,val_dataloader,lablesall= get_ds()
    image_model,image_processor =get_image_model()
    model_audio,processor_audio =get_audio_model()
    text_model,text_Tokenizer =get_text_model()
    image_layerno_1,image_layerno_2= get_hidden_layer()
  
    print("Training dataset size ", len(train_dataloader )*config['batch_size'] )  

    model = get_model().to(device)

     # Tensorboard
    writer = SummaryWriter(config['experiment_name'])


   # optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)
    value_opt = torch.optim.Adam(params = model.parameters(), lr = config['lr'])

    # If the user specified a model to preload before training, load it
    initial_epoch = 0
    global_step = 0
    preload = config['preload']
    model_filename = latest_weights_file_path(config) if preload == 'latest' else get_weights_file_path(config, preload) if preload else None
    if model_filename:
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)
        model.load_state_dict(state['model_state_dict'])
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']
    else:
        print('No model to preload, starting from scratch')

    #loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)   
    


   # abc=torch.rand(1, 768).to(device)
    print("Layers ",config["num_layers"])
    
    for epoch in range(initial_epoch, config['num_epochs']):
        #print("epoch",epoch)
        
        torch.cuda.empty_cache()
        #train_dataset_both ,val_dataset_both ,train_dataloader,val_dataloader= get_ds()
        model.train()
        batch_size=config['batch_size'] 
  
        batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")

        for batch in batch_iterator:
              
########################  image KQV extraction 
            

            paths=batch["path"]
            image_file = Dataset.from_dict({"image": paths}).cast_column("image", Image())  
            #dataset = Dataset.from_dict({"image": ["path/to/image_1", "path/to/image_2", ..., "path/to/image_n"]}).cast_column("image", Image())
            #dataset[0]["image"]
            #print("maithri ranga1",image_file)
            iend = config['batch_size'] 
            image = image_processor(images=image_file[0:iend]["image"], return_tensors="pt").to(device)

           # image = image_processor(images=batch['path'], return_tensors="pt").to(device)

            image_model=image_model.to(device)
            image_outputs = image_model(**image)
            image_last_hidden_state = image_outputs.last_hidden_state
            image_pooled_output = image_outputs.pooler_output  
           # print("image_pooled_output",image_pooled_output.shape)
           # batch_nummber+= 1
         
            image_layear_output=image_outputs.hidden_states[image_layerno_1],image_outputs.hidden_states[image_layerno_2]

            

            label=batch["label"]

           
            #print("imageddddddddddddddddddddddd",label,flabel)
            inputs_text = text_Tokenizer(text=label, padding=True, return_tensors="pt").to(device)
            
            text_model= text_model.to(device)
            text_outputs = text_model(**inputs_text)
            
            text_pooled_output = text_outputs.pooler_output
           # print("imageddddddddddddddddddddddd",text_pooled_output.shape)
            text_layear_output=text_outputs.hidden_states[image_layerno_1],text_outputs.hidden_states[image_layerno_2]
            


            #text model parameters 
            text_query_bias=text_model.encoder.layer[image_layerno_1].attention.attn.q.bias,text_model.encoder.layer[image_layerno_2].attention.attn.q.bias 
            text_key_bias=text_model.encoder.layer[image_layerno_1].attention.attn.k.bias,text_model.encoder.layer[image_layerno_2].attention.attn.k.bias
            text_value_bias=text_model.encoder.layer[image_layerno_1].attention.attn.v.bias,text_model.encoder.layer[image_layerno_2].attention.attn.v.bias
            _start = 0
            _end = config["d_model"]
            text_query_weight=text_model.encoder.layer[image_layerno_1].attention.attn.q.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.q.weight[_start:_end, :]
            text_key_weight=text_model.encoder.layer[image_layerno_1].attention.attn.k.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.k.weight[_start:_end, :]
            text_value_weight=text_model.encoder.layer[image_layerno_1].attention.attn.v.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.v.weight[_start:_end, :]




            #image model parameters 
            image_query_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.q_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.q_proj.bias
            image_key_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.k_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.k_proj.bias
            image_value_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.v_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.v_proj.bias

            image_query_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.q_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.q_proj.weight[_start:_end, :]
            image_key_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.k_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.k_proj.weight[_start:_end, :]
            image_value_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.v_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.v_proj.weight[_start:_end, :]
            
            #proj_output=model(config['d_model'], config['ffn_hidden'], config['num_heads'], config['drop_prob'], config['num_layers'], config['kn_vocab_size'], model_audio, image_model, outputs_audio, image_outputs)
            proj_output=model(batch_size,text_query_bias,text_key_bias,text_value_bias,text_query_weight,text_key_weight,text_value_weight,image_query_bias,image_key_bias,image_value_bias, image_query_weight,image_key_weight,image_value_weight, 
                              text_layear_output, image_layear_output,image_pooled_output).to(device)
           # out_image = Transformer(audio_query_bias,audio_key_bias,audio_value_bias,audio_query_weight,audio_key_weight,audio_value_weight, image_model, audio_layear_output, image_layear_output)
            text_pooled_output=text_pooled_output/text_pooled_output.norm(dim=-1, keepdim=True)

      
            #image_pooled_output=torch.squeeze(image_pooled_output,dim=0).to(device)
  
            proj_output=proj_output/proj_output.norm(dim=-1, keepdim=True)
            image_last_hidden_state=image_last_hidden_state/image_last_hidden_state.norm(dim=-1, keepdim=True)
            image_pooled_output=image_pooled_output/image_pooled_output.norm(dim=-1, keepdim=True)

            loss = loss_fn(proj_output.view(-1,768), text_pooled_output .view(-1,768))   # compare with pooled_output image to audio

            


            batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

            # Log the loss
            writer.add_scalar('train loss', loss.item(), global_step)
            writer.flush()

            # Backpropagate the loss
            loss.backward()

            # Update the weights
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

        
            global_step += 1
            



        model_filename = get_weights_file_path(config, f"{epoch:02d}")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)

if __name__ == '__main__':
    warnings.filterwarnings("ignore")
    config = get_config()
    train_model(config)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
config = get_config()
train_dataset_both ,val_dataset_both,labels_all = get_ds()
print("Layers ",config["num_layers"])
print("Training dataset size ", len(val_dataset_both ))
#val_dataset_both2 =val_dataset_both.shard(num_shards=2, index=0)
model = get_model().to(device)
Accuracy = []
for number in range(0, 40, 1):
    if number < 10: 
        str(number)
        number= "0"+ str(number)
    else:
        number=str(number)
    cwd2 = "tmodel_" + number + ".pt"
    cwd = "weights"
    os.path.join(cwd,cwd2)



    # Load the pretrained weights
    model_filename = latest_weights_file_path(config)
    model_filename =os.path.join(cwd,cwd2)
    print(model_filename)
    state = torch.load(model_filename)
    model.load_state_dict(state['model_state_dict'])
    tAccuracy = run_validation(model, val_dataset_both,labels_all, device,  lambda msg: print(msg), 0, None, num_examples=2500)
    Accuracy.append(tAccuracy)
    print("Merge",tAccuracy)
    with open('output.txt', 'a') as testwritefile:
        testwritefile.write(str(number))
        testwritefile.write("layer accuracy ")
        testwritefile.write(str(tAccuracy ))
        testwritefile.write("\n ")
    print(Accuracy)
max_value_index = np.argmax(Accuracy)
Maximum_Accuracy = Accuracy[max_value_index]
print("Maximum_Accuracy:", Maximum_Accuracy ,"Maximum Accuracy epoch:",max_value_index )
with open('output.txt', 'a') as testwritefile:
        testwritefile.write("Maximum_Accuracy:")
        testwritefile.write(str(Maximum_Accuracy))
        testwritefile.write("Maximum Accuracy epoch:")
        testwritefile.write(str(max_value_index))